# Approach: Depth First Search


### Intuition

We can also use a depth-first search (DFS) traversal to detect the nodes that lead to a cycle, i.e., unsafe nodes.

In DFS, we use a recursive function to explore nodes as far as possible along each branch. Upon reaching the end of a branch, we backtrack to the previous node and continue exploring the next branches.

Once we encounter an unvisited node, we will take one of its neighbor nodes (if exists) as the next node on this branch. Recursively call the function to take the next node as the 'starting node' and solve the subproblem.

A node remains in the DFS recursion stack until all of its branches (all nodes in its subtree) have not been explored. When we have examined all of a node's branches, i.e. visited all of the nodes in its subtree, the node is removed from the DFS recursive stack.

If you are new to Depth First Search, please see our Leetcode Explore Card for more information on it!

To find the unsafe nodes, we must first recognize a cycle in the graph. If we find a cycle, we will mark all of the nodes in the cycle as unsafe and then go back and mark all of the nodes that led to this cycle as unsafe. Let's find a cycle first.

If the graph has a cycle, we must have a back edge connecting a node to one of its ancestors while traversing nodes in the DFS manner.

Let's think how we can establish whether or not a node's neighbor is an ancestor when navigating from one node to another.

If the neighboring node has not yet been visited, it cannot be an ancestor (it is a child node).

Otherwise, if a neighboring node is visited, it may or may not be an ancestor. If the neighboring node is an ancestor, i.e. there is a back edge, it means that we visited this ancestor node first in the DFS traversal, then visited and explored some other nodes, and eventually visited a node that connects back to the ancestor node. As we are still exploring the ancestor node's subtree while iterating over this path, hence this node must be in the current DFS recursive stack.

However, if a neighboring node is visited but not in the recursion stack, it signifies we have previously explored that node in a different branch, and it does not form a cycle in the current branch.

As a result, to detect the cycle we must keep track of the visited nodes (like in a normal DFS) and also the nodes in the function's recursion call stack for DFS traversal. The nodes in the stack store the current path that we are on. There is a cycle in the graph if a node is reached that is already in the recursion stack. We use a boolean array inStack of length n to track which nodes are in the call stack so we can check if a node exists in O(1). Note that this inStack array is emulating the call stack that the computer is using under the hood to execute recursion. We mark an unvisited node in inStack when we make a recursive call to it and then unmark it when we return from that call.

Now that we've identified the cycle, let's look for the unsafe nodes. When we get a cycle, all of the nodes in the recursion stack either form or lead to a cycle. If we start a DFS traversal from node 1 in a graph 1 -> 2 -> 3 -> 4 -> 2, nodes 2, 3, and 4 form a cycle. When we discovered this cycle, node 1 was also in the stack. So, when we have a cycle, all the nodes in the recursion stack are unsafe since they form or lead to a loop.

In addition to detecting cycles, we can use the same inStack array to store the unsafe nodes. We do not unmark any of the unsafe nodes from inStack to keep track of them. When any node has an outgoing edge to any of the unsafe nodes, we can immediately return the DFS call for node without unmarking it from inStack, i.e, we do not perform inStack[node] = false. This is because if any neighbor of node is marked inStack, it signifies that either neighbor and node are part of a cycle or neighbor is a previously detected unsafe node. In both the cases, node is also an unsafe node and hence we return the DFS call without unmarking node from inStack.

We only unmark a node from inStack, if we have explored all of its branches and no branch leads to an unsafe node.

### Algorithm

Here, we can use the input graph as the adjacency list adj

Create two boolean arrays, visit and inStack, each of size n. The visit array keeps track of visited nodes and inStack keeps track of nodes that are currently in the ongoing DFS stack. It will help us to detect a cycle in the graph and the unsafe nodes.
For each node we begin a DFS traversal. We implement the dfs method which takes four parameters: an integer node from which the current traversal begins, adj, visit, and inStack. It returns a boolean indicating whether node is unsafe. We perform the following in this method:
If node is already present in inStack, either we just got a cycle or a previously detected unsafe node. We return true in this case as the node is unsafe.
If node is already visited (but not in inStack), we return false because we already visited this node and didn't find it as unsafe node. It is a safe node.
We mark node as visited and also mark it in inStack (inStack[node] = true).
We iterate over all the outgoing edges of node and for each neighbor, we recursively call dfs(neighbor, adj, visit, inStack). If we get a cycle from neighbor (or neighbor is a previously detected unsafe node), we return true without unmarking node in inStack.
After we have processed all the outgoing edges of node, we mark inStack[node] = false to mark node as safe. We return false.
Create an answer array safeNodes of size n. Iterate over all the nodes from 0 to n - 1 and add all the safe nodes in safeNodes, i.e., the nodes with inStack[node] == false.
Return safeNodes.

In [5]:
from typing import List

class Solution:
    def dfs(self, node, adj, visit, inStack):
        # If the node is already in the stack, we have a cycle.
        if inStack[node]:
            return True
        if visit[node]:
            return False
        # Mark the current node as visited and part of current recursion stack.
        visit[node] = True
        inStack[node] = True
        for neighbor in adj[node]:
            if self.dfs(neighbor, adj, visit, inStack):
                return True
        # Remove the node from the stack.
        inStack[node] = False
        return False

    def eventualSafeNodes(self, graph: List[List[int]]) -> List[int]:
        n = len(graph)

        visit = [False] * n
        inStack = [False] * n

        for i in range(n):
            self.dfs(i, graph, visit, inStack)

        safeNodes = []
        for i in range(n):
            if not inStack[i]:
                safeNodes.append(i)

        return safeNodes

In [6]:
graph_1 = [[1,2],[2,3],[5],[0],[5],[],[]]
print(Solution().eventualSafeNodes(graph_1))

graph_2 = [[1,2],[2,3],[5],[0],[5],[2],[]]
print(Solution().eventualSafeNodes(graph_2))

graph_3 = [[1,2,3,4],[1,2],[3,4],[0,4],[]]
print(Solution().eventualSafeNodes(graph_3))

[2, 4, 5, 6]
[6]
[4]
